# Example of using LLM-based information retrieval for recommendation task

Run in google colaboratory

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import sys
import os

repo_path = '/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec'
sys.path.append(repo_path)

## Install requirements

In [4]:
!pip install -q -r '{repo_path}/requirements/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

## Add config

In [36]:
import os

config_dict = {
    "csv_args": {"delimiter": "\t"},
    "source_column": "item_id:token",
    "search_kwargs": {"k": 20},
    "data_path": os.path.join(repo_path, "datasets"),
    "load_col": {
        "inter": ["user_id", "item_id", "rating", "timestamp"],
        "item": ["item_id", "title"],
    },
    "text_col": "title",
    "MAX_ITEM_LIST_LENGTH": 10,
    "eval_args": {"split": {"LS": "valid_and_test"}, "order": "TO", "mode": "full"},
    "repeatable": True,
    "loss_type": "CE",
    "train_batch_size": 100,
    "eval_batch_size": 8,
    "valid_metric": "NDCG@10",
    "metrics": ["Recall", "NDCG"],
    "topk": [1, 5, 10, 20],
    "train_neg_sample_args": None,
}

## Get dataset and config

In [37]:
from llm4rec.tasks import RecBoleRetrievalRecommender
from llm4rec.dataset import RecboleSeqDataset
from llm4rec.trainer import PipelineTrainer
from recbole.data.utils import data_preparation
from recbole.config import Config
from recbole.model.abstract_recommender import AbstractRecommender
import os
import torch

model_cls = RecBoleRetrievalRecommender
dataset_name = 'amazon-books'

config = Config(model=model_cls, dataset=dataset_name,
            config_dict=config_dict)

dataset = RecboleSeqDataset(config)
_, _, test_data = data_preparation(config, dataset)

## Case 1: open source information retrieval using HuggingFace Sentence Transformers model

In [38]:
model = model_cls(config=config,
                embeddings=None,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                source_column=config['source_column'],
                csv_args=config['csv_args'],
                text_splitter_args=dict(chunk_size=1000, chunk_overlap=0),
                search_type="similarity",
                search_kwargs=config['search_kwargs'],
                emb_model_name="all-MiniLM-L6-v2",
                emb_model_kwargs={"device":"cuda:0" if torch.cuda.is_available() else "cpu"},
                dataset=dataset)

trainer = PipelineTrainer(config, model)
test_result = trainer.evaluate(test_data, show_progress=config['show_progress'])

Evaluate   : 100%|██████████████████████████████████████████████████| 25/25 [00:01<00:00, 13.13it/s]


In [39]:
test_result

OrderedDict([('recall@1', 0.01),
             ('recall@5', 0.055),
             ('recall@10', 0.07),
             ('recall@20', 0.08),
             ('ndcg@1', 0.01),
             ('ndcg@5', 0.0311),
             ('ndcg@10', 0.0363),
             ('ndcg@20', 0.0385)])

## Case 2: OpenAI information retrieval using OpenAIEmbeddings

In [28]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

path_to_openai_env = os.path.join(repo_path, "openai.env")
load_dotenv(path_to_openai_env)

openai_api_key = os.environ.get("API_KEY")
embeddings_model = OpenAIEmbeddings(
            openai_api_key=openai_api_key, model="text-embedding-ada-002"
        )
model = model_cls(config=config,
                embeddings=embeddings_model,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                source_column=config['source_column'],
                csv_args=config['csv_args'],
                text_splitter_args=dict(chunk_size=1000, chunk_overlap=0),
                search_type="similarity",
                search_kwargs=config['search_kwargs'],
                emb_model_name="",
                emb_model_kwargs=None,
                dataset=dataset)

trainer = PipelineTrainer(config, model)
test_result = trainer.evaluate(test_data, show_progress=config['show_progress'])

Evaluate   : 100%|██████████████████████████████████████████████████| 25/25 [01:30<00:00,  3.63s/it]


In [30]:
test_result

OrderedDict([('recall@1', 0.035),
             ('recall@5', 0.07),
             ('recall@10', 0.085),
             ('recall@20', 0.125),
             ('ndcg@1', 0.035),
             ('ndcg@5', 0.0529),
             ('ndcg@10', 0.0578),
             ('ndcg@20', 0.0683)])